In [1]:
import json
import re
import time
import requests
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')
from random import choice

In [2]:
current_datetime = datetime.now().strftime("%d-%m-%Y")
month_now = datetime.now().month
day_now = datetime.now().day

In [3]:
way = os.getcwd()

# Cookies and Headers для парсинга скидки и цены

In [4]:
cookies_discount = {
    'tmr_lvid': '286726132090325c59b36aa6b5e4aeaa',
    'tmr_lvidTS': '1692163620235',
    '_ym_uid': '1692163620496112993',
    '_ym_d': '1692163620',
    'city_id': '118',
    'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
    'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_subdomain': 'spb',
    '_tt_enable_cookie': '1',
    '_ttp': 'tbKEEA3IkGrgdKpaT6-HRNf0K8h',
    'ccart': 'off',
    'uxs_uid': '898ab7e0-3bf5-11ee-8efe-f5b770db12a1',
    'loggedin': '%D0%AE%D1%80%D0%B8%D0%B9',
    'cseid': 'f4935df6bdac0029832e99cef94a5bc2',
    'sessionid': 'nr24mlml3ze25rk734gkkmgx3yv9ou7x',
    'authorization': 'true',
    'mindboxDeviceUUID': 'c254fa3f-cfcd-4146-9778-65be21c6a4df',
    'directCrm-session': '%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D',
    'auid': '651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA',
    '_ga_G74SVHM1ZG': 'GS1.2.1700824600.19.0.1700824600.60.0.0',
    '_ga_GRDJMMMRHQ': 'GS1.2.1700824600.33.0.1700824601.59.0.0',
    '_ga_GRSMPDQ6SE': 'GS1.2.1700824598.88.1.1700825516.60.0.0',
    '_ga_M9ZHPEJXBM': 'GS1.2.1700824599.88.1.1700825517.60.0.0',
    'qr-go-to-the-app-banner': '1',
    'sncid': 'b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams',
    'clt': 'new_old',
    '_ym_isad': '2',
    '_gid': 'GA1.2.2131018395.1702274408',
    'city_auto_popup_shown': '1',
    '_ym_visorc': 'b',
    '_ga': 'GA1.2.333284150.1692163620',
    '_ga_HJNSJ6NG5J': 'GS1.1.1702276831.120.1.1702280271.20.0.0',
    'digi_uc': 'W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
}

headers_discount = {
    'AB-TESTS': '{}',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'tmr_lvid=286726132090325c59b36aa6b5e4aeaa; tmr_lvidTS=1692163620235; _ym_uid=1692163620496112993; _ym_d=1692163620; city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; _tt_enable_cookie=1; _ttp=tbKEEA3IkGrgdKpaT6-HRNf0K8h; ccart=off; uxs_uid=898ab7e0-3bf5-11ee-8efe-f5b770db12a1; loggedin=%D0%AE%D1%80%D0%B8%D0%B9; cseid=f4935df6bdac0029832e99cef94a5bc2; sessionid=nr24mlml3ze25rk734gkkmgx3yv9ou7x; authorization=true; mindboxDeviceUUID=c254fa3f-cfcd-4146-9778-65be21c6a4df; directCrm-session=%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D; auid=651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA; _ga_G74SVHM1ZG=GS1.2.1700824600.19.0.1700824600.60.0.0; _ga_GRDJMMMRHQ=GS1.2.1700824600.33.0.1700824601.59.0.0; _ga_GRSMPDQ6SE=GS1.2.1700824598.88.1.1700825516.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1700824599.88.1.1700825517.60.0.0; qr-go-to-the-app-banner=1; sncid=b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams; clt=new_old; _ym_isad=2; _gid=GA1.2.2131018395.1702274408; city_auto_popup_shown=1; _ym_visorc=b; _ga=GA1.2.333284150.1692163620; _ga_HJNSJ6NG5J=GS1.1.1702276831.120.1.1702280271.20.0.0; digi_uc=W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
    'Origin': 'https://spb.sunlight.net',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'SunlightFrontendApp',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# params = {
#     'article': '35517',
#     'city_id': '118',
# }

# response = requests.get(
#     'https://api.sunlight.net/v6/catalog/products/card/purchase/',
#     params=params,
#     cookies=cookies_discount,
#     headers=headers_discount,
# )

# Сбор скидок и цен

In [5]:
def get_personal_price(table):
    data_price = []
    for art in tqdm(range(len(table['article']))):
        # prx = choice(proxy_list)
        params = {
            'article': table['article'][art],
            'city_id': '118'
        }
        try:
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
        )
        except:
            time.sleep(6)
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        try:
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        except:
            time.sleep(6)
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        discount = []
        try:
            price_old = price_json.json()['content']['default_price']['base_sum']
        except:
            price_old = np.nan
        try:
            price = price_json.json()['content']['default_price'][('result_sum')]
        except:
            price = np.nan
        try:
            price_personal = price_json.json()['content']['default_price']['personal_block']['value']
        except:
            price_personal = np.nan

        try:
            for i in range(len(discount_json.json()['content']['nameplates'])):
                discount.append(discount_json.json()['content']['nameplates'][i]['text']['label'])
        except:
            discount.append(np.nan)
            
        data_price.append({
            'article': table['article'][art],
            'price_old': price_old,
            'price': price,
            'price_personal': price_personal,
            'discount': discount

        })
    return data_price

# Сбор цены, названия, группы, артикула

In [6]:
def get_sunlight(soup):
    try:
        price = soup.find(class_="supreme-product-card__price-discount-price").text
        price = re.sub(r'\D', '', str(price))
    except:
        price = np.nan
    try:
        name_product = soup.find('h1', class_="supreme-product-card__info-name-title").text
    except:
        name_product = np.nan
    try:
        group_product = soup.find_all('span', {'property': 'name'})[1].text
    except:
        group_product = np.nan
    try:
        article_product = soup.find_all('div', class_="supreme-product-card__info-name-article")[0].find('span').text
        article_product = re.sub('\D', '', str(article_product))
    except:
        article_product = np.nan
    return price, name_product, group_product, article_product

# Сбор всех параметров словарем

In [7]:
def get_params(soup):
    names = []
    values = []
    leight = len(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'}))
    list_names = soup.find_all('div', {'class': 'supreme-product-card-description__item-title'})[-leight:]
    for i in range(leight):
        try:
            nam = list_names[i].text
            names.append(nam.replace('\n', '').strip())
        except:
            names.append(np.nan)
        try:
            values.append(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'})[i].text)
        except:
            values.append(np.nan)
    sunlight_params = dict(zip(names, values))
    return sunlight_params

# Сбор ссылок

In [8]:
def get_links(url_sunlight):
    home = 'https://sunlight.net'
    links = []
    pages_max = 250
    pages = int(pages_max + 1)
    for i in tqdm(range(1, pages)):
        url = f'{url_sunlight}&page={i}'
        resp = requests.get(url, headers={'User-Agent': UserAgent().chrome}, verify=False, timeout=70)
        soup = BeautifulSoup(resp.text, 'html.parser')
        for link in soup.find_all('div', class_="cl-item js-cl-item"):
            links.append(home + link.find('a', href=True)['href'])
    links = list(map(lambda x: x.replace('#product-reviews', ''), links))
    return links

# Общий сбор данных

In [9]:
def sunlight_parser(links):
    data = []
    click = 0
    for link_pars in tqdm(links):
        # prx = choice(proxy_list)
        try:
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        except:
            time.sleep(6)
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        good = BeautifulSoup(resp.text, 'lxml')
        data.append({"date": current_datetime,
                    "competitor": "Sunlight",
                    "name": get_sunlight(good)[1],
                    "group": get_sunlight(good)[2],
                    "params": get_params(good),
                    "price": get_sunlight(good)[0],
                    "article": get_sunlight(good)[3],
                    'url': link_pars
                    })
        if click == 50:
            click = 0
            time.sleep(0.5)
        else:
            continue
    data = pd.DataFrame(data)
    return data

In [10]:
links_sunlight = get_links('https://spb.sunlight.net/catalog/?proba=2494')

  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [04:02<00:00,  1.03it/s]


In [11]:
b = pd.DataFrame(links_sunlight)

In [12]:
b = b.drop_duplicates()

In [13]:
b = b[0].tolist()

In [14]:
slices = 100
chunks = np.array_split(b, slices)

In [15]:
n = 50
for i in range(n, slices):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = sunlight_parser(chunks[i])
    df.to_csv(rf'{way}\parsed_data\sunlight_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 51 части ссылок


100%|██████████| 140/140 [02:45<00:00,  1.18s/it]


начало парсинга 52 части ссылок


100%|██████████| 140/140 [02:14<00:00,  1.04it/s]


начало парсинга 53 части ссылок


...

100%|██████████| 139/139 [02:00<00:00,  1.15it/s]


начало парсинга 99 части ссылок


100%|██████████| 139/139 [02:02<00:00,  1.13it/s]


начало парсинга 100 части ссылок


100%|██████████| 139/139 [01:58<00:00,  1.17it/s]


In [16]:
os.chdir(rf'{way}\parsed_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

100

In [17]:
data = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([df, data], ignore_index=True)

In [18]:
data

,date,competitor,name,group,params,price,article,url
0,18-06-2024,Sunlight,NaN,Браслеты,"{'Модель': '73520-B5Y-01', 'Материал изделия':...",219990.0,NaN,https://sunlight.net/catalog/bracelets_989819....
1,18-06-2024,Sunlight,NaN,Браслеты,"{'Модель': '73751-B5Y-01', 'Материал изделия':...",699990.0,NaN,https://sunlight.net/catalog/bracelets_989802....
2,18-06-2024,Sunlight,NaN,Браслеты,"{'Модель': '30139-B5W-02', 'Материал изделия':...",239990.0,NaN,https://sunlight.net/catalog/bracelets_989418....
3,18-06-2024,Sunlight,NaN,Браслеты,"{'Модель': '30141-B5Y-01', 'Материал изделия':...",244990.0,NaN,https://sunlight.net/catalog/bracelets_989416....
4,18-06-2024,Sunlight,NaN,Браслеты,"{'Модель': '30141-B5W-01', 'Материал изделия':...",239990.0,NaN,https://sunlight.net/catalog/bracelets_989415....
...,...,...,...,...,...,...,...,...
13955,18-06-2024,Sunlight,\n Золотые серьги с агатом и бриллиантами\n,Серьги,"{'Модель': '71794-C5R-01', 'Материал изделия':...",97990.0,310221.0,https://sunlight.net/catalog/earring_310221.html
13956,18-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '30906.1*', 'Материал изделия': 'Ро...",5490.0,91078.0,https://sunlight.net/catalog/earring_91078.html
13957,18-06-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '12623.1*', 'Материал изделия': 'Бе...",9990.0,88213.0,https://sunlight.net/catalog/pendants_88213.html
13958,18-06-2024,Sunlight,\n Золотые серьги с изумрудами и бриллианта...,Серьги,"{'Модель': '70990-C5R-01', 'Материал изделия':...",54990.0,253164.0,https://sunlight.net/catalog/earring_253164.html


In [19]:
data.to_csv(rf'{way}\data_without_price.csv', index = False)

In [20]:
# data = pd.read_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\data_without_price.csv')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13960 entries, 0 to 13959
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        13960 non-null  object 
 1   competitor  13960 non-null  object 
 2   name        8632 non-null   object 
 3   group       13619 non-null  object 
 4   params      13960 non-null  object 
 5   price       13619 non-null  float64
 6   article     8632 non-null   float64
 7   url         13960 non-null  object 
dtypes: float64(2), object(6)
memory usage: 872.6+ KB


In [22]:
data.duplicated().sum()

0

In [23]:
data = data.drop_duplicates()

In [24]:
data.duplicated().sum()

0

In [25]:
data['article'].isna().sum()

5328

In [26]:
data = data.dropna()

In [27]:
data['article'] = data['article'].astype('int') 

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8632 entries, 1390 to 13959
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        8632 non-null   object 
 1   competitor  8632 non-null   object 
 2   name        8632 non-null   object 
 3   group       8632 non-null   object 
 4   params      8632 non-null   object 
 5   price       8632 non-null   float64
 6   article     8632 non-null   int32  
 7   url         8632 non-null   object 
dtypes: float64(1), int32(1), object(6)
memory usage: 573.2+ KB


In [29]:
test = np.array_split(data, 20)

In [30]:
test[0]

,date,competitor,name,group,params,price,article,url
1390,18-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '221223*', 'Материал изделия': 'Роз...",32990.0,87003,https://sunlight.net/catalog/earring_87003.html
1391,18-06-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '73586-K5Y-01', 'Материал изделия':...",71990.0,990408,https://sunlight.net/catalog/ring_990408.html
1392,18-06-2024,Sunlight,\n Золотая подвеска\n,Подвески,"{'Модель': '00-63-0170-00*', 'Материал изделия...",28990.0,991939,https://sunlight.net/catalog/pendants_991939.html
1393,18-06-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '71221-С5R-01', 'Материал изделия':...",45990.0,275421,https://sunlight.net/catalog/earring_275421.html
1394,18-06-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '2139881010*', 'Материал изделия': ...",23990.0,349502,https://sunlight.net/catalog/earring_349502.html
...,...,...,...,...,...,...,...,...
5292,18-06-2024,Sunlight,\n Золотое колье с бриллиантами\n,Колье,"{'Модель': '72553-H5R-01', 'Материал изделия':...",48990.0,329827,https://sunlight.net/catalog/necklace_329827.html
5298,18-06-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '04-62-0153-09*', 'Материал изделия...",10490.0,348316,https://sunlight.net/catalog/earring_348316.html
5301,18-06-2024,Sunlight,\n Золотое кольцо с фианитами и ситаллами\n,Кольца,"{'Модель': '100-1645_prst-212', 'Материал изде...",36990.0,355389,https://sunlight.net/catalog/ring_355389.html
5304,18-06-2024,Sunlight,\n Золотое кольцо с ониксом и бриллиантами\n,Кольца,"{'Модель': '90095-K5Y-01', 'Покрытие': 'с элем...",57990.0,99429,https://sunlight.net/catalog/ring_99429.html


In [31]:
n = 0
for i in range(n, 20):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = get_personal_price(test[i].reset_index())
    df = pd.DataFrame(df)
    df.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts\\' + f'sunlight_all_data_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 432/432 [05:01<00:00,  1.43it/s]


начало парсинга 2 части ссылок


100%|██████████| 432/432 [04:42<00:00,  1.53it/s]


начало парсинга 3 части ссылок


...

100%|██████████| 431/431 [04:33<00:00,  1.57it/s]


начало парсинга 19 части ссылок


100%|██████████| 431/431 [04:28<00:00,  1.60it/s]


начало парсинга 20 части ссылок


100%|██████████| 431/431 [04:51<00:00,  1.48it/s]


In [32]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

20

In [33]:
data

,date,competitor,name,group,params,price,article,url
1390,18-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '221223*', 'Материал изделия': 'Роз...",32990.0,87003,https://sunlight.net/catalog/earring_87003.html
1391,18-06-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '73586-K5Y-01', 'Материал изделия':...",71990.0,990408,https://sunlight.net/catalog/ring_990408.html
1392,18-06-2024,Sunlight,\n Золотая подвеска\n,Подвески,"{'Модель': '00-63-0170-00*', 'Материал изделия...",28990.0,991939,https://sunlight.net/catalog/pendants_991939.html
1393,18-06-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '71221-С5R-01', 'Материал изделия':...",45990.0,275421,https://sunlight.net/catalog/earring_275421.html
1394,18-06-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '2139881010*', 'Материал изделия': ...",23990.0,349502,https://sunlight.net/catalog/earring_349502.html
...,...,...,...,...,...,...,...,...
13955,18-06-2024,Sunlight,\n Золотые серьги с агатом и бриллиантами\n,Серьги,"{'Модель': '71794-C5R-01', 'Материал изделия':...",97990.0,310221,https://sunlight.net/catalog/earring_310221.html
13956,18-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '30906.1*', 'Материал изделия': 'Ро...",5490.0,91078,https://sunlight.net/catalog/earring_91078.html
13957,18-06-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '12623.1*', 'Материал изделия': 'Бе...",9990.0,88213,https://sunlight.net/catalog/pendants_88213.html
13958,18-06-2024,Sunlight,\n Золотые серьги с изумрудами и бриллианта...,Серьги,"{'Модель': '70990-C5R-01', 'Материал изделия':...",54990.0,253164,https://sunlight.net/catalog/earring_253164.html


In [34]:
data_discount = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data_discount = pd.concat([df, data_discount], ignore_index=True)

In [35]:
data_discount

,article,price_old,price,price_personal,discount
0,353478,55 980 ₽,27 990 ₽,16 794 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 20.06']"
1,300010,37 980 ₽,18 990 ₽,11 394 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
2,353665,46 220 ₽,23 110 ₽,13 866 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
3,991701,239 980 ₽,119 990 ₽,91 990 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 20.06']"
4,241109,173 300 ₽,51 990 ₽,31 194 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
...,...,...,...,...,...
8627,329827,97 980 ₽,48 990 ₽,29 394 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
8628,348316,20 980 ₽,10 490 ₽,6 294 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 20.06']"
8629,355389,73 980 ₽,36 990 ₽,22 194 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 20.06']"
8630,99429,115 980 ₽,57 990 ₽,34 794 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"


In [36]:
data = data.merge(data_discount, on='article', how='left')

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632 entries, 0 to 8631
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            8632 non-null   object 
 1   competitor      8632 non-null   object 
 2   name            8632 non-null   object 
 3   group           8632 non-null   object 
 4   params          8632 non-null   object 
 5   price_x         8632 non-null   float64
 6   article         8632 non-null   int32  
 7   url             8632 non-null   object 
 8   price_old       7975 non-null   object 
 9   price_y         8632 non-null   object 
 10  price_personal  7979 non-null   object 
 11  discount        8632 non-null   object 
dtypes: float64(1), int32(1), object(10)
memory usage: 775.7+ KB


In [38]:
data['discount'].value_counts()

discount
['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']                                                                             4777
['Цены 8 марта! Ещё -10%, код: МАРТ до 20.06']                                                                            1632
['Ещё -10% при покупке двух, код: ДВА', 'Ещё -15% при покупке трех, код: ТРИ']                                            1185
['ХИТ-цена']                                                                                                               637
['Цены 8 марта! Ещё -15%, код: МАРТ до 20.06']                                                                             250
['Цены снижены! Ещё -15%, код: СЕГОДНЯ']                                                                                    83
['Ещё -10% при покупке двух, код: ДВА', 'Ещё -15% при покупке трех, код: ТРИ', 'Цены снижены! Ещё -15%, код: СЕГОДНЯ']      21
[nan]                                                                                                 

In [39]:
data.head(20)

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,18-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '221223*', 'Материал изделия': 'Роз...",32990.0,87003,https://sunlight.net/catalog/earring_87003.html,65 980 ₽,32 990 ₽,19 794 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
1,18-06-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '73586-K5Y-01', 'Материал изделия':...",71990.0,990408,https://sunlight.net/catalog/ring_990408.html,143 980 ₽,71 990 ₽,43 990 ₽,"['Ещё -10% при покупке двух, код: ДВА', 'Ещё -..."
2,18-06-2024,Sunlight,\n Золотая подвеска\n,Подвески,"{'Модель': '00-63-0170-00*', 'Материал изделия...",28990.0,991939,https://sunlight.net/catalog/pendants_991939.html,57 980 ₽,28 990 ₽,17 394 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
3,18-06-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '71221-С5R-01', 'Материал изделия':...",45990.0,275421,https://sunlight.net/catalog/earring_275421.html,91 980 ₽,45 990 ₽,27 594 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
4,18-06-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '2139881010*', 'Материал изделия': ...",23990.0,349502,https://sunlight.net/catalog/earring_349502.html,47 980 ₽,23 990 ₽,14 394 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
5,18-06-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '127910*', 'Материал изделия': 'Роз...",21990.0,345832,https://sunlight.net/catalog/earring_345832.html,43 980 ₽,21 990 ₽,13 194 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
6,18-06-2024,Sunlight,\n Золотое кольцо c бриллиантами\n,Кольца,"{'Модель': '2804-K5R-02', 'Материал изделия': ...",69990.0,17945,https://sunlight.net/catalog/ring_17945.html,139 980 ₽,69 990 ₽,41 994 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
7,18-06-2024,Sunlight,\n Золотые серьги с сапфирами и бриллиантам...,Серьги,"{'Модель': '73601-C5W-01', 'Материал изделия':...",114990.0,985649,https://sunlight.net/catalog/earring_985649.html,229 980 ₽,114 990 ₽,86 990 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
8,18-06-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '70486-C5R-01', 'Материал изделия':...",109990.0,106332,https://sunlight.net/catalog/earring_106332.html,219 980 ₽,109 990 ₽,81 990 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
9,18-06-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '1125294/8*', 'Материал изделия': '...",7490.0,335890,https://sunlight.net/catalog/earring_335890.html,14 980 ₽,7 490 ₽,4 494 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 20.06']"


In [40]:
data.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\\' + f'sunlight_all_{day_now}_{month_now}_{datetime.today().year}.csv', index=False)

In [41]:
previous = pd.read_csv(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv')

In [42]:
data = pd.concat([previous, data], ignore_index=True)

In [43]:
data

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,21-05-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '71409-K5W-02', 'Материал изделия':...",99990.0,985936,https://sunlight.net/catalog/ring_985936.html,199 980 ₽,99 990 ₽,82 990 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
1,21-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73353-B5R-01', 'Материал изделия':...",399990.0,985849,https://sunlight.net/catalog/bracelets_985849....,799 980 ₽,399 990 ₽,382 990 ₽,"['Второе бесплатно, код:1=2 до 27.05']"
2,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '01Ц032974Д*', 'Материал изделия': ...",133820.0,343470,https://sunlight.net/catalog/chains_343470.html,267 640 ₽,133 820 ₽,116 820 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
3,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '01Ц012974Д*', 'Материал изделия': ...",148990.0,343469,https://sunlight.net/catalog/chains_343469.html,297 980 ₽,148 990 ₽,131 990 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
4,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '45-0029-10*', 'Материал изделия': ...",106200.0,343447,https://sunlight.net/catalog/chains_343447.html,212 400 ₽,106 200 ₽,89 200 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
...,...,...,...,...,...,...,...,...,...,...,...,...
80354,18-06-2024,Sunlight,\n Золотые серьги с агатом и бриллиантами\n,Серьги,"{'Модель': '71794-C5R-01', 'Материал изделия':...",97990.0,310221,https://sunlight.net/catalog/earring_310221.html,195 980 ₽,97 990 ₽,69 990 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
80355,18-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '30906.1*', 'Материал изделия': 'Ро...",5490.0,91078,https://sunlight.net/catalog/earring_91078.html,NaN,5 490 ₽,NaN,['ХИТ-цена']
80356,18-06-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '12623.1*', 'Материал изделия': 'Бе...",9990.0,88213,https://sunlight.net/catalog/pendants_88213.html,19 980 ₽,9 990 ₽,5 994 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"
80357,18-06-2024,Sunlight,\n Золотые серьги с изумрудами и бриллианта...,Серьги,"{'Модель': '70990-C5R-01', 'Материал изделия':...",54990.0,253164,https://sunlight.net/catalog/earring_253164.html,183 300 ₽,54 990 ₽,32 994 ₽,"['Цены 8 марта! Ещё -5%, код: МАРТ до 20.06']"


In [44]:
data.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv', index=False)

In [45]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

In [46]:
delete_files_in_folder('C:\\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data')
delete_files_in_folder('C:\\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')